# 06_Base_sumarizada

## 01_import_bibliotecas

In [ ]:
# === Geração da base consolidada para análise regional por estado + cluster ===

import pandas as pd
import numpy as np
from datetime import datetime
import joblib
from io import BytesIO
import requests
from sklearn.preprocessing import StandardScaler

## 02_Carga_de_dados

In [ ]:
candidatos_sum = 'https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/01_fontes/arquivos_decision/fontes_tratadas/04_dados_candidatos.parquet'

In [ ]:
df_2 = pd.read_parquet(candidatos_sum)

## 03_Tratamento

In [ ]:
# === 2. Calcular idade com base em data_nascimento ===
hoje = pd.to_datetime("today")
df_2["data_nascimento"] = pd.to_datetime(df_2["data_nascimento"], errors="coerce")
df_2["idade"] = df_2["data_nascimento"].apply(lambda x: hoje.year - x.year if pd.notnull(x) else np.nan)


In [ ]:
# 2.2. Mapear nomes de estado para siglas
mapa_estados = {
    "acre": "AC", "alagoas": "AL", "amapá": "AP", "amazonas": "AM", "bahia": "BA",
    "ceará": "CE", "distrito federal": "DF", "espírito santo": "ES", "goiás": "GO",
    "maranhão": "MA", "mato grosso": "MT", "mato grosso do sul": "MS", "minas gerais": "MG",
    "pará": "PA", "paraíba": "PB", "paraná": "PR", "pernambuco": "PE", "piauí": "PI",
    "rio de janeiro": "RJ", "rio grande do norte": "RN", "rio grande do sul": "RS",
    "rondônia": "RO", "roraima": "RR", "santa catarina": "SC", "são paulo": "SP",
    "sergipe": "SE", "tocantins": "TO"
}
df_2["estado"] = df_2["endereco"].str.lower().map(mapa_estados)

In [ ]:
# === 3. Carregar modelo e colunas usadas ===

modelo = joblib.load(BytesIO(requests.get("https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/03_modelos/modelo_kmeans.joblib").content))

colunas_modelo = joblib.load(BytesIO(requests.get(f"https://github.com/Kinrider/tech_challenge_5/raw/refs/heads/main/03_modelos/colunas_usadas.joblib").content))

In [ ]:
# Tratamento de nivel_educacional
mapa_educacional = {
    'ensino fundamental': 1,
    'ensino médio': 2,
    'ensino superior': 2,
    'superior incompleto': 2,
    'pós-graduação ou mais': 3,
    'não identificado': 0,
    None: 0
}
df_2['nivel_educacional'] = df_2['nivel_educacional'].map(mapa_educacional).fillna(0).astype(int)

In [ ]:
# Conversão de variável booleana experiencia_sap
df_2['experiencia_sap'] = df_2['experiencia_sap'].astype(int)

In [ ]:
# One-hot encoding para variáveis categóricas
colunas_categoricas = ['categoria_profissional', 'subarea_profissional', 'nivel_hierarquico']
df_2 = pd.get_dummies(df_2, columns=colunas_categoricas, drop_first=True)

In [ ]:
# Escalonamento de variáveis contínuas
variaveis_continuas = ['remuneracao_zscore', 'tempo_experiencia_anos', 'quantidade_experiencias']
scaler = StandardScaler()
df_2[variaveis_continuas] = scaler.fit_transform(df_2[variaveis_continuas])

In [ ]:
# Tratar valores ausentes
colunas_com_nan = df_2.columns[df_2.isnull().any()].tolist()
for col in colunas_com_nan:
    df_2[f"tem_{col}"] = df_2[col].notnull().astype(int)
    df_2[col] = df_2[col].fillna(-9999)

### Entrada_do_modelo

In [ ]:
df_2

In [ ]:
# Remover colunas não utilizadas
colunas_remover = [
    'data_nascimento',
    'id_candidato',
    'remuneracao_mensal_brl',
    'endereco',
    'estado',
]

df_model = df_2.drop(columns=colunas_remover, errors='ignore')

In [ ]:
# === 4. Preparar features de entrada ===
# df_model = df_2.copy()

# Preencher flags e sentinelas
for col in colunas_modelo:
    if col.startswith("tem_"):
        original = col.replace("tem_", "")
        df_model[col] = df_model[original].notnull().astype(int)
        df_model[original] = df_model[original].fillna(-9999)

# Garantir todas as colunas
for col in colunas_modelo:
    if col not in df_model.columns:
        df_model[col] = 0

In [ ]:
# === 5. Prever cluster para cada candidato ===
X_input = df_model[colunas_modelo]
df_2["cluster"] = modelo.predict(X_input)

In [ ]:
# 2.10. Faixa etária
bins = [0, 24, 34, 44, 54, 64, 120]
labels = ["até 24", "25 a 34", "35 a 44", "45 a 54", "55 a 64", "65+"]
df_2["faixa_etaria"] = pd.cut(df_2["idade"], bins=bins, labels=labels, right=True)

In [ ]:
df_2.head()

In [ ]:
# === 6. Agregar por estado + cluster ===
df_sumarizado = df_2.groupby(["estado", "cluster", "faixa_etaria"]).agg(
    candidatos=("id_candidato", "count"),
    escolaridade_media=("nivel_educacional", "mean"),
    perc_com_sap=("experiencia_sap", "mean"),
    remuneracao_media=("remuneracao_mensal_brl", lambda x: x.replace(-9999, np.nan).mean()),
    experiencias_media=("quantidade_experiencias", "mean")
).reset_index()

# Arredondamentos
df_sumarizado["escolaridade_media"] = df_sumarizado["escolaridade_media"].round(2)
df_sumarizado["perc_com_sap"] = (df_sumarizado["perc_com_sap"] * 100).round(1)
df_sumarizado["remuneracao_media"] = df_sumarizado["remuneracao_media"].round(2)
df_sumarizado["experiencias_media"] = df_sumarizado["experiencias_media"].round(2)

## 04_Salvando_base

In [ ]:
df_sumarizado.head(25)

In [ ]:
Caminho = "C:\\Users\\pedro\\Documents\\Área de Trabalho\\tech_challenge_5\\01_fontes\\arquivos_decision\\fontes_tratadas\\"

In [ ]:
df_sumarizado["estado"] = df_sumarizado["estado"].astype(str)

In [ ]:
# === 7. Exportar para Parquet e Excel ===
df_sumarizado.to_parquet(Caminho + "05_dados_sumarizados.parquet", index=False)

df_sumarizado.to_excel(Caminho + "05_dados_sumarizados.xlsx", index=False)